# ETL II: data extraction with web scrapping

<br>

<p align="center">
<img width=80 src="https://media.giphy.com/media/KAq5w47R9rmTuvWOWa/giphy.gif"  class="center" />
</p>
    
***

## Scrapping data from the web

Sometimes we have data displayed on the web and that website don't have a download function or an api to access to the content, so the option we have is to read the content "As a user" and get that content.

To do this we'll use the urllib library and beautiful soup.

Imagine we need to extract the filmography of Anne Hathaway and we found this information in wikipedia (https://es.wikipedia.org/wiki/Anne_Hathaway), let's see the process to get the data.

### Getting the raw data

To get the raw data we need to make a call to the website we want to get. The best way to not be blocked is to use our firefox headers.

In [1]:
from urllib.request import Request, urlopen

webpage = 'https://es.wikipedia.org/wiki/Anne_Hathaway'

# 1. Get HTML from the web:
req = Request(webpage, headers={'User-Agent': 'Mozilla/5.0'})
raw_web = urlopen(req, timeout=10).read()

With this, we have the whole HTML of the website in the `raw_web`
variable.

*Annoying question: did you understand the `timeout` parameter?*

Here are the first 400 characters:

In [2]:
raw_web[:400]

b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-cu'

Essentially, it is the same as viewing the page source on a website. This can be done by right-clicking and selecting "View Page Source," or by using the keyboard shortcut Ctrl + U.

*Another annoying question: didn't we did this in the previous notebook?*

2.2 GET request...

No...?

Below "Check this easy example:" ...

So... What are we doing?

### Looking for the data we need

Now we have the content, we need to clean everything we don't want. 

We'll use beautful soup library to get the table data (docs:
<https://www.crummy.com/software/BeautifulSoup/bs4/doc/>):

In [11]:
from bs4 import BeautifulSoup

# 2. We parse the web using Beautiful Soup
soup = BeautifulSoup(raw_web, 'html.parser')

# If you try to print the whole soup, the output is very long. 
# Try it uncommenting the following line:
# print(soup)

# Let's print some lines
# Convert the BeautifulSoup object to a string
soup_str = str(soup)

# Split the string into lines
lines = soup_str.split('\n')

# Print the first 6 lines
for line in lines[:6]:
    print(line)

<!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-disabled vector-toc-available" dir="ltr" lang="es">
<head>
<meta charset="utf-8"/>
<title>Anne Hathaway - Wikipedia, la enciclopedia libre</title>


First, we need to determine the location of the desired data. The easiest way to do this is by opening the website in a new tab, right-clicking on the content (such as a table) we want to save, and selecting "Inspect".

Once we do that, we can see that the content of interest is nested within a `<table>` tag, like the following:

`<table class="wikitable sortable jquery-tablesorter" ...bla.bla.bla... ><table>`


In [14]:
tables = soup.find_all('table')
# print(tables)

# Extract the first table from the BeautifulSoup object
first_table = soup.find('table')
# Print the entire first table
# print(first_table)

# Print a short description or specific attribute (e.g., class attribute)
if first_table:
    print(f"First table found with class: {first_table.get('class')}")
else:
    print("No table found.")

First table found with class: ['infobox', 'biography', 'vcard']


With the command above, we have extracted all the tables on the website. We can access each of them by their index position within the array in which they are stored. 

So, if we want to know how many tables there are, what can we do?

In [15]:
len(tables)

6

We can attempt another extraction method by restricting the results to the specific class we observed during inspection:

In [16]:
# 3. We retrieve the tables of interest
tables = soup.find_all('table', attrs={"class": "wikitable sortable"})
len(tables)

2

Now we have narrowed down to two tables, and we can explore them. Let's focus on the first table for now.

To begin, we need to extract the headers:

In [17]:
# 4.a. We retrieve the header
table_header = tables[0].find_all('th')[1:] # We skip the first header as we don't want it
print(table_header)

[<th align="center">Año
</th>, <th align="center">Título original
</th>, <th align="center">Papel
</th>, <th align="center">Notas
</th>]


and rows of the table:

In [18]:
# 4.b. We retrieve the rows of the table
table_rows = tables[0].find_all('tr')[2:]
table_rows

[<tr>
 <td rowspan="2">2001
 </td>
 <td><i><a href="/wiki/The_Princess_Diaries" title="The Princess Diaries">The Princess Diaries</a></i>
 </td>
 <td><a class="mw-redirect" href="/wiki/Mia_Thermopolis" title="Mia Thermopolis">Mia Thermopolis</a>
 </td>
 <td>
 </td></tr>,
 <tr>
 <td><i><a href="/wiki/The_Other_Side_of_Heaven" title="The Other Side of Heaven">The Other Side of Heaven</a></i>
 </td>
 <td>Jean Sabin
 </td>
 <td>
 </td></tr>,
 <tr>
 <td rowspan="2">2002
 </td>
 <td><i><a href="/wiki/Neko_no_Ongaeshi" title="Neko no Ongaeshi">Neko no Ongaeshi</a></i>
 </td>
 <td>Haru Yoshioka
 </td>
 <td>Voz
 </td></tr>,
 <tr>
 <td><i><a href="/wiki/Nicholas_Nickleby_(pel%C3%ADcula_de_2002)" title="Nicholas Nickleby (película de 2002)">Nicholas Nickleby</a></i>
 </td>
 <td>Madeline Bray
 </td>
 <td>
 </td></tr>,
 <tr>
 <td rowspan="2">2004
 </td>
 <td><i><a href="/wiki/Ella_Enchanted" title="Ella Enchanted">Ella Enchanted</a></i>
 </td>
 <td>Ella de Frell
 </td>
 <td>
 </td></tr>,
 <tr>
 <td

Once we have obtained the data, we need to clean the HTML to extract only the content, which is the plain text, and discard the HTML tags. 

Oh, the beloved .strip() method comes to the rescue!

What do you have on your hands?

Are you sure you are understanding the objects and their types that we have at our disposal?

In [19]:
print(type(table_header).__name__)

list


In [20]:
len(table_rows)

42

In [21]:
table_rows[0]

<tr>
<td rowspan="2">2001
</td>
<td><i><a href="/wiki/The_Princess_Diaries" title="The Princess Diaries">The Princess Diaries</a></i>
</td>
<td><a class="mw-redirect" href="/wiki/Mia_Thermopolis" title="Mia Thermopolis">Mia Thermopolis</a>
</td>
<td>
</td></tr>

I trust you...

Let us create a script for cleaning the df:

In [22]:
# 5. Table cleaning

theads = []
tdata = []

# Extract each title from the table
for head in table_header:
    theads.append(head.text.strip()) # With the strip() method we can delete the new line character (\n)

year = 0    # Initialize a variable to store the desired year
            # Since not all rows have a year, we save the ones at disposal
           
for row in table_rows[:-2]: # Iterate over each row of the table, excluding the last two

    rdata = [] # Empty array to store the content of each row

    for col in row: # Iterate over each column of the row, as my data now has two dimensions,
                    # we need to iterate through both
        
        colTxt  = col.text.strip() # Extract the text from the column
        
        if colTxt != '': # Discard empty columns
            
            if len(colTxt) == 4: # If the column has 4 characters, it signifies the year
                year = colTxt
                
            rdata.append(colTxt) # Store all columns as elements of the list
        
    # Fix the entries without a year
    if len(rdata[0]) != 4: # If the length of the first column is not 4, the year is missing 

        # Let us choose a straightforward path, jeje
        rdata.insert(0, year)    
    
    if len(rdata) == 3: # If the row does not have a notes field, we add an empty value. 
        
        # Dou you understand why?
        
        # Adding an empty value to the row when it does not have a notes field ensures consistency
        # in the data structure. This can be beneficial for subsequent data processing or analysis, 
        # as it facilitates handling and helps avoid potential issues with missing values.
        rdata.append('')
        
    tdata.append(rdata)

In [23]:
tdata[:13]

[['2001', 'The Princess Diaries', 'Mia Thermopolis', ''],
 ['2001', 'The Other Side of Heaven', 'Jean Sabin', ''],
 ['2002', 'Neko no Ongaeshi', 'Haru Yoshioka', 'Voz'],
 ['2002', 'Nicholas Nickleby', 'Madeline Bray', ''],
 ['2004', 'Ella Enchanted', 'Ella de Frell', ''],
 ['2004', 'The Princess Diaries 2: Royal Engagement', 'Mia Thermopolis', ''],
 ['2005', 'Hoodwinked!', 'Caperucita Roja', 'Voz'],
 ['2005', 'Havoc', 'Allison Lang', ''],
 ['2005',
  'Brokeback Mountain',
  'Lureen Newsome Twist',
  'Nominación: Premio SAG al mejor reparto'],
 ['2006', 'The Devil Wears Prada', 'Andrea Sachs', ''],
 ['2007', 'Becoming Jane', 'Jane Austen', ''],
 ['2008', 'Get Smart', 'Agente 99', ''],
 ['2008', "Get Smart's Bruce and Lloyd: Out of Control", 'Cortometraje', '']]

In [24]:
import pandas as pd
# 6. Create dataframe
result = pd.DataFrame(tdata, columns=theads)
result

,Año,Título original,Papel,Notas
0,2001,The Princess Diaries,Mia Thermopolis,
1,2001,The Other Side of Heaven,Jean Sabin,
2,2002,Neko no Ongaeshi,Haru Yoshioka,Voz
3,2002,Nicholas Nickleby,Madeline Bray,
4,2004,Ella Enchanted,Ella de Frell,
5,2004,The Princess Diaries 2: Royal Engagement,Mia Thermopolis,
6,2005,Hoodwinked!,Caperucita Roja,Voz
7,2005,Havoc,Allison Lang,
8,2005,Brokeback Mountain,Lureen Newsome Twist,Nominación: Premio SAG al mejor reparto
9,2006,The Devil Wears Prada,Andrea Sachs,


Here we have the full table from Wikipedia!!  

Meeejjjj....

Don't always believe what you're told...

<br>

As you have seen, I implemented a different method for adding an element at the beginning of a list than the one shown in the video ;)


**Observations and disadvantages:**

- There are many ways to add an element at the beginning of a list. Here are some: https://www.geeksforgeeks.org/python-perform-append-at-beginning-of-list/

- Another option is to build the list dynamically. First, add the year and then the rest of the content.

- Of course, I won't do it...

- If a word within the table content has 4 characters, then the logic fails! Capicci?

- Solution: We could use RegEx and add a condition that the string starts with the character "2". This way, we would only consider strings with a year ;)

<br>

To be honest, I wouldn't have written all this code... I'm a lazy person... 

I would have just kept the rows that I can handle quickly and easily, for example, the ones with a well-defined year. For instance:

In [26]:
theads = []
trows =[]
i=1

for col in table_header:
    theads.append(col.text.strip())

for row in table_rows:
    content = []

    for col in row:
        content.append(col.text.strip())
        
    try:
        content = [content[1], content[3], content[5], content[7]] # We select only the columns we need
        trows.append(content)

        #print(i,'-If the try is successfull, the row has', len(row), 'features')
        #i += 1

    except:
        error = 'ignore' # We add this line just to ignore the error that is normal in our case

        #print('If an exception is raised, the row has', len(row), 'features')
        #print(row)


result = pd.DataFrame(trows, columns=theads)
result

,Año,Título original,Papel,Notas
0,2001,The Princess Diaries,Mia Thermopolis,
1,2002,Neko no Ongaeshi,Haru Yoshioka,Voz
2,2004,Ella Enchanted,Ella de Frell,
3,2005,Hoodwinked!,Caperucita Roja,Voz
4,2006,The Devil Wears Prada,Andrea Sachs,
5,2007,Becoming Jane,Jane Austen,
6,2008,Get Smart,Agente 99,
7,2009,Guerra de novias,Emma Allen,
8,2010,Valentine's Day,Elizabeth «Liz» Curran,
9,2011,Río,Perla,Voz


**13 lines of code!!**

**How concise!**

💜

Now we have extracted some useful data, the data that is easily accessible!

However, every time I get lazy, something clever comes to my mind... 

I noticed that the useful rows had a length of 8, while many of the non-useful rows had a length of 6. In the code above, I have left the print statements commented for you to play with!

With this advantage of the dataset, I had proceed:

In [27]:
theads = []
trows =[]
i = 1

for col in table_header:
    theads.append(col.text.strip()) 
    
for row in table_rows:
    
    content = [] 
    
    for col in row: 
        content.append(col.text.strip())
    
    # Only 
    try:
        if len(content) == 8:
            contents = [content[1], content[3], content[5], content[7]] # Direct rows
            year = content[1]
        
        else:
            contents = [year, content[1], content[3], content[5]] # This rows can be easily handle also ;)
        
        #print(i,'-',contents)
        i +=1
        trows.append(contents)
    except:
        error = 'ignore'
        print('IGNORE the row of length', len(row)) # Let us print the useless rows

IGNORE the row of length 4
IGNORE the row of length 2
IGNORE the row of length 2


In [28]:
result = pd.DataFrame(trows, columns=theads)
result

,Año,Título original,Papel,Notas
0,2001,The Princess Diaries,Mia Thermopolis,
1,2001,The Other Side of Heaven,Jean Sabin,
2,2002,Neko no Ongaeshi,Haru Yoshioka,Voz
3,2002,Nicholas Nickleby,Madeline Bray,
4,2004,Ella Enchanted,Ella de Frell,
5,2004,The Princess Diaries 2: Royal Engagement,Mia Thermopolis,
6,2005,Hoodwinked!,Caperucita Roja,Voz
7,2005,Havoc,Allison Lang,
8,2005,Brokeback Mountain,Lureen Newsome Twist,Nominación: Premio SAG al mejor reparto
9,2006,The Devil Wears Prada,Andrea Sachs,


Now we have an almost complete dataset from Wikipedia, which is good enough for me.

<br>

Some nitpickers might argue that there is one missing row...

To them, I would respond that the original code from the LiveCoding session had a misinterpretation of one of the lines, so I skipped it 😏

If you're interested, you can check which row is incorrect to understand what is happening. Throughout the code, I left hints about which line it is. Did you see them? 🤡

Otherwise, let's save this data for later use with the "open" command:

In [29]:
with open('sources/Anne_Hathaway.txt', 'w') as file:
    
    content = '' # We create an empty content
    
    content = content + ','.join(theads) + '\n' # We add the heads as the first line
    
    for row in trows:
        content = content + ','.join(row)+'\n' # This adds each row as a line        
    
    file.write(content) # This writes the content in the file

This is what you need to extract the most types of data you'll work with. 

Remember also, you can get data from other sources, for example an SQL database as you saw in the SQL module.

You'll need to pay attention to every source you want to connect to because they could be all different and have some particularities you'll need to solve. But don't worry, practice and experience will let you solve any case.